# ドラクエ 6 - 戦闘終了時にお宝を入手する確率を計算する

本稿ではオリジナル版ドラクエ 6 における一度の戦闘終了時の、敵側からお宝を入手する確率の計算法について考察する。
また、Jupyter Notebook のレッスンを兼ねるものでもある。

## 基本的事実
### C2B091: お宝入手シーケンス
味方パーティーが敵方のお宝を入手したことになる事象を全て列挙する。実際のプログラムは大体このような処理を行う：

0. メンバー 1 が盗んだ
+ メンバー 1 が盗まず、かつメンバー 2 が盗んだ
+ メンバー 1 も 2 も盗まず、かつメンバー 3 が盗んだ
+ メンバー 1 も 2 も 3 も盗まず、かつメンバー 4 が盗んだ
+ 魔物が宝箱を落とした
+ [else] 何もお宝が得られなかった

説明の都合上、魔物が宝箱を落としたケースを先に議論する。
それから、味方メンバーによる盗みのケースを議論する。

### C2B13F: 落とされたことによるお宝入手確率
魔物が宝箱を落とす確率は 8 段階に定義されている。この確率の基になる定数がプログラムのアドレス C2B1C3 に定義されている。
以降の議論では、便宜上これらの定数をお宝定数を呼ぶことにする。

```text
                        // COMMENT
C2/B1C3:	0000        // +1 =    1
C2/B1C5:	0700        // +1 =    8
C2/B1C7:	0F00        // +1 =   16
C2/B1C9:	1F00        // +1 =   32
C2/B1CB:	3F00        // +1 =   64
C2/B1CD:	7F00        // +1 =  128
C2/B1CF:	FF00        // +1 =  256
C2/B1D1:	FF0F        // +1 = 4096
```

ここでは詳しく説明しないが、固定した魔物に対して（個体ではなくクラスにより）この定数が一意に決まる。
例えば必ず「ちいさなメダル」を入手できる「なげきのきょじん」は 0x0000 であり、
なかなか「はかいのてっきゅう」を入手できないことで知られる「エビルフランケン」は 0x0FFF となる。

敵方の魔物が宝箱を落とす確率を $p_d$ とおこう。この値をどう評価すればよいのかを見ていく。

とあるサブルーチンで次のような振る舞いをするものがある：

* 受け取った 2 バイトの数に 1 を加えた数 `a` を
* 範囲 $[0, 0x10000)$ にある数のいずれかランダムに決めた値だけ乗じて 4 バイト長の数を生成し、
* その上位 2 バイトを 2 バイト長の数として返す。

プログラムコードでは、このサブルーチンの引数に上の表から取った定数を与えて呼び出し、戻り値がゼロでないことを以って宝箱を落としていったことにしている。平たく言うと、この定数に対応する確率表を下のように表現できる：

$$p_d \in \left\{1, \frac{1}{8}, \frac{1}{16}, \frac{1}{32}, \frac{1}{64}, \frac{1}{128}, \frac{1}{256}, \frac{1}{4096}\right\}$$

分かりやすく言うと、
（味方パーティーに盗賊が一人もいなくて）倒した魔物にお宝が設定されているときには、
魔物に対応する上述の確率の基になる定数の設定もされていて、
ここに記した 8 通りのいずれかの確率で入手イベントが発生する。

### C2B09A: 盗んだことによるお宝入手確率
次に、一人の味方メンバーが敵方の魔物から宝物を盗む確率 $p_t$ を考える。
議論を簡単にするため、次のことを仮定しておく：

* 戦闘終了時に馬車の外に味方パーティーメンバーが 4 名いる
* どのメンバーも職業が盗賊であり、職業レベルがいずれも $a$ である
* どのメンバーも戦闘終了時に「しに」「まひ」「ねむり」のいずれの状態でもない

現在就いている職業が盗賊であり、
かつ盗賊レベルが $a$ であるようなパーティーメンバー一人が対象の魔物からお宝を盗む確率
$p_t$ を調べよう。

* とあるサブルーチンにお宝定数を 4 倍した値を渡して、
* 範囲 $[0, 0x10000)$ にある数のいずれかランダムに決めた値だけ乗じて 4 バイト長の数を生成し、
* その上位 2 バイトを 2 バイト長の数として戻り値として得る。

* この乱数的数と職業レベルを比較し、職業レベルのほうが大きければ盗んだことにして処理を終了する。
* 乱数のほうが大きければ、次のメンバーについて同様の乱数生成および大小比較を行う。誰も盗んだことにならなければ処理を終了する。

というシーケンスなので、

* メンバーの盗賊としての職業レベルを $a$, 
* 魔物のお宝定数を $c$

とおくと、

$$p_t = \min \left\{ \frac{a}{4c + 1}, 1 \right\}$$

として扱ってよい。ここで $a = 1, .., 8$ であり、$c$ は後ほど述べる定数配列 C2B1C3 の魔物クラスから決まるいずれかの値である。
$c = 0$ のときには盗賊レベルが 1 であっても盗んだことになるので、右辺を 1 として扱いたいので便宜上 $\min$ を入れてあるだけだ。

## 基本的な定数を計算する

これまでに出てきた定数を Python を利用してコード化する。とりあえず NumPy をインポートしよう。

In [1]:
import numpy as np
np.set_printoptions(formatter={'int':hex})

お宝定数の配列 C2B1C3 を配列オブジェクト `C` として表現する。

In [2]:
C = np.array([0x0000,
              0x0007,
              0x000F,
              0x001F,
              0x003F,
              0x007F,
              0x00FF,
              0x0FFF,], dtype=int)
C

array([0x0, 0x7, 0xf, 0x1f, 0x3f, 0x7f, 0xff, 0xfff])

この `C` から直ちに魔物が宝箱を落とす確率を得られる。これを `P_d` としよう。

In [3]:
P_d = 1 / (C + 1); P_d

array([  1.00000000e+00,   1.25000000e-01,   6.25000000e-02,
         3.12500000e-02,   1.56250000e-02,   7.81250000e-03,
         3.90625000e-03,   2.44140625e-04])

盗賊レベルが 1 である盗賊メンバーがお宝を盗む確率を `P_t` とする。
これは `min` を使わずに済み、次のように定義できる。

In [4]:
P_t = 1 / (4 * C + 1); P_t

array([  1.00000000e+00,   3.44827586e-02,   1.63934426e-02,
         8.00000000e-03,   3.95256917e-03,   1.96463654e-03,
         9.79431929e-04,   6.10463342e-05])

ここで、盗賊レベルが `n` である盗賊メンバーが盗む確率は、この値を `n` 倍して 1 と比べて小さいほうを取ることで得られる。Python のコードとしては `P_t[P_t > 1] = 1` なり `np.clip(P_t, 0, 1)` なりすればよいだろう。

##  「はかいのてっきゅう」獲得確率計算例
盗賊レベルが最大であるような盗賊職のメンバー 4 人からなるパーティーがエビルフランケンと戦い、
戦闘を通常勝利で終了した際に「はかいのてっきゅう」が得られる確率を計算する例を示す。
ここで

* `p_d` をエビルフランケンが宝箱を落とす確率、
* `p_t` を盗賊レベル $a = 8$ のメンバーの一人がお宝定数 $c = 0x0FFF$ で宝箱を落とす魔物からモノを盗む確率、
* リスト `P` を上述の事象それぞれの確率

と置く。Python で簡単な計算をして各確率を評価すると次のようになる。
まず職業レベル `a` とお宝定数 `c` を定義する：

In [5]:
a, c = 8, C[7]

`p_d` と `p_t` をそれぞれ定義する：

In [6]:
p_d = P_d[7]
p_t = min(8 * P_t[7], 1)
p_d, p_t

(0.000244140625, 0.00048837067334106586)

### 戦闘終了時に鉄球が入手できる確率を計算したい場合

事象 1. から事象 5. の確率を全部加算して、その値を 1 から引いたほうが早い。
戦闘終了後、盗賊の誰かが盗んだか、エビルフランケンが宝箱を落としたかのいずれかが起こる確率を `p` と置く。
すると次のように確率の値を計算できる：

In [7]:
p = 1 - (1 - p_t)**4 * (1 - p_d)
p, 1 - p

(0.0021957161735115527, 0.99780428382648845)

盗賊レベル最大の 4 人パーティーでエビルフランケンと戦ったときに「はかいのてっきゅう」を獲得する確率は、
概算で 0.22% 程度の値であることがわかった。

## 戦闘終了時にお宝を入手できる確率表を作成する

「はかいのてっきゅう」以外にも、より一般的な次の状況における戦闘終了時のお宝入手確率を表にまとめておく。つまり：

* お宝定数の全 8 通りについて計算する
* パーティー内のアクティブな盗賊人数が 0 から 4 までの場合を全て計算する
  ただし、そのような盗賊の盗賊レベルは一律最大値 8 とする（ここまで場合分けするには紙幅が足りない）
  
NumPy の配列計算機能を利用すると、コード量がかなり軽くなる：

In [8]:
num_members = np.arange(0, 5) # 0 to 4

prob_d = 1 / (C + 1)
prob_t = np.clip(a / (4 * C + 1), 0, 1) # a == 8

PT, N = np.meshgrid(prob_t, num_members)
PD, N = np.meshgrid(prob_d, num_members)

prob_table = 1 - ((1 - PT) ** N * (1 - PD))

確率表のデータが得られたので、各成分を百分率の形式で出力してみよう。
表の横軸と縦軸は魔物のお宝定数と有効盗賊人数とにそれぞれ対応する：

In [9]:
percent_formatter = lambda x: '{:>8.3%}'.format(x)
np.set_printoptions(formatter={'float_kind':percent_formatter})
print(prob_table)

[[100.000%  12.500%   6.250%   3.125%   1.562%   0.781%   0.391%   0.024%]
 [100.000%  36.638%  18.545%   9.325%   4.675%   2.341%   1.171%   0.073%]
 [100.000%  54.117%  29.228%  15.128%   7.689%   3.876%   1.945%   0.122%]
 [100.000%  66.774%  38.509%  20.560%  10.608%   5.386%   2.714%   0.171%]
 [100.000%  75.940%  46.574%  25.644%  13.435%   6.873%   3.476%   0.220%]]


## 参考文献
* [ドラクエ6　盗みの確率](http://dq6.org/nusumi.html)
* [【盗賊】 - ドラゴンクエスト大辞典を作ろうぜ！！第三版 Wiki*](http://wikiwiki.jp/dqdic3rd/?%A1%DA%C5%F0%C2%B1%A1%DB)
* [解析・計算式 | ドラゴンクエスト6 攻略・解析](http://gamecentergx.at-ninja.jp/dq6/analyze.html)
* [(SFC) ドラゴンクエスト6 - 盗むの確率](http://gamecentergx.blog.fc2.com/blog-entry-1301.html)